# Modelo de Ising en 3D

Autor: David Duran<br>
Año: 2023

In [21]:
# %pip install mayavi
# %pip install pyqt5

In [22]:
# Importar paquetes
import numpy as np
import matplotlib.pyplot as plt
import numba
from numba import njit
from mayavi import mlab
from tqdm.notebook import tqdm # Barra de progreso

In [23]:
# Lattice
tamaño_red = 4

# Red tridimensional
rejilla = np.random.choice([-1, 1], size=(tamaño_red, tamaño_red, tamaño_red))

In [24]:
# Algoritmo de Metropolis para la red 3D
@numba.njit(nogil=True)
def metropolis(rejilla, pasos, BJ, energia):
    # Copiar la rejilla para no modificar la original
    rejilla = rejilla.copy()

    # Crear arreglos para almacenar la magnetización y la energía
    magnetizacion = np.zeros(pasos - 1)
    energia_total = np.zeros(pasos - 1)

    # Bucle para actualizar la rejilla (loop to update the lattice)
    for t in range(0, pasos - 1):
        # Seleccionar un punto aleatorio en la rejilla y cambiar el spin
        x = np.random.randint(0, tamaño_red)
        y = np.random.randint(0, tamaño_red)
        z = np.random.randint(0, tamaño_red)

        spin_inicial = rejilla[x, y, z]
        spin_propuesto = spin_inicial * -1

        # Calcular el cambio de energía
        energia_inicial = 0
        energia_final = 0
        vecinos = [
            (x - 1, y, z), (x + 1, y, z),
            (x, y - 1, z), (x, y + 1, z),
            (x, y, z - 1), (x, y, z + 1)
        ]
        for nx, ny, nz in vecinos:
            if 0 <= nx < tamaño_red and 0 <= ny < tamaño_red and 0 <= nz < tamaño_red:
                energia_inicial += -spin_inicial * rejilla[nx, ny, nz]
                energia_final += -spin_propuesto * rejilla[nx, ny, nz]

        # Actualizar el estado con las probabilidades designadas
        delta_energia = energia_final - energia_inicial
        if (delta_energia > 0) * (np.random.random() < np.exp(-BJ * delta_energia)):
            rejilla[x, y, z] = spin_propuesto
            energia += delta_energia
        elif delta_energia <= 0:
            rejilla[x, y, z] = spin_propuesto
            energia += delta_energia

        # Actualizar la magnetización y la energía total
        magnetizacion[t] = rejilla.sum()
        energia_total[t] = energia

    return {'rejilla': rejilla, 'magnetizacion': magnetizacion, 'energia_total': energia_total}

In [25]:
# Ejecutar la simulación
# magnetizacion, energia_total = metropolis(rejilla, 100000, 0.2, 0)
magnetizacion, energia_total = metropolis(rejilla.astype(np.int64), 100000, 0.2, 0)

# Graficar la magnetización y la energía total
fig, ejes = plt.subplots(1, 2, figsize=(12, 4))
eje = ejes[0]
eje.plot(magnetizacion / tamaño_red ** 3)
eje.set_xlabel('Pasos del algoritmo')
eje.set_ylabel('Magnetización promedio')
eje.grid()

eje = ejes[1]
eje.plot(energia_total)
eje.set_xlabel('Pasos del algoritmo')
eje.set_ylabel('Energía')
eje.grid()

fig.tight_layout()
fig.suptitle('Evolución de la magnetización promedio y la energía', y=1.07, size=18)
plt.show()

TypeError: cannot convert native LiteralStrKey[Dict]({Literal[str](rejilla): array(int64, 3d, C), Literal[str](magnetizacion): array(float64, 1d, C), Literal[str](energia_total): array(float64, 1d, C)}) to Python object

In [ ]:
# %pip install mayavi
# %pip install pyqt5

In [ ]:
def visualize_spins(lattice, t, output_dir):
    mlab.figure(bgcolor=(1, 1, 1), size=(400, 308))

    for i in range(lattice.shape[0]):
        for j in range(lattice.shape[1]):
            for k in range(lattice.shape[2]):
                spin = lattice[i, j, k]
                color = (0, 0, 1) if spin == 1 else (1, 0, 0)
                mlab.quiver3d(i, j, k, 0, 0, spin, color=color, mode='arrow', scale_factor=0.5)

    mlab.savefig(f"{output_dir}/frame_{t:04d}.png")
    mlab.close()

# visualize_spins(rejilla)

In [ ]:
def simulate_and_visualize(rejilla, pasos, BJ, energia, visualization_interval=1000, output_dir="."):
    rejilla = rejilla.astype(np.int64)  # Convert to 64-bit integers
    num_iterations = pasos // visualization_interval
    with tqdm(total=num_iterations) as pbar:
        for t in range(num_iterations):
            results = metropolis(rejilla, visualization_interval, BJ, energia)
            rejilla = results['rejilla']
            visualize_spins(rejilla, t, output_dir)
            pbar.update()

simulate_and_visualize(rejilla, 100000, 0.2, 0, visualization_interval=1000)